In [ ]:
import sys,os
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import datasets,models
import torchvision.transforms as T

from PIL import Image as image1

from torch.optim import lr_scheduler

import matplotlib.pyplot as plt
import time
import os
import copy

from sklearn.model_selection import train_test_split

#conv2d(input_channels, output_channels, kernel_size)

In [ ]:
!pip install wandb
import wandb
wandb.login()

In [ ]:
# Data compression

import cv2
import glob
from PIL import Image
ghghgh
trainingData = []
trainingLabels = []

#folders = ['Animalia', 'Aves', 'Amphibia', 'Arachnida', 'Plantae', 'Fungi', 'Mollusca', 'Mammalia', 'Insecta', 'Reptilia']
folders = ['Mammalia', 'Insecta']
classes = { folders[i] : i for i in range(0,len(folders))}
#folders = folders[:3]

resize = T.Compose([
    T.ToPILImage(),           
    T.Resize(224)])

for name in folders:
  print(name)
  start_time = datetime.now()
  imdir = '/content/drive/MyDrive/nature_12K/inaturalist_12K/train/'+name+'/'
  ext = ['png', 'jpg', 'gif']    # Add image formats here

  files = []
  [files.extend(glob.glob(imdir + '*.' + e)) for e in ext]

  images = [cv2.imread(file) for file in files]
  
  count=1
  for img in images:
    rimg = resize(img)
    rimg.save('/content/drive/MyDrive/nature_12K/resized/train/'+name+'/'+str(count)+'.jpg')
    count+=1

  end_time = datetime.now()
  print('Duration: {}'.format(end_time - start_time))

print("=========== Testing =====================")
testingData = []
testingLabels = []
for name in folders:
  print(name)
  start_time = datetime.now()
  imdir = '/content/drive/MyDrive/nature_12K/inaturalist_12K/val/'+name+'/'
  ext = ['png', 'jpg', 'gif']    # Add image formats here

  files = []
  [files.extend(glob.glob(imdir + '*.' + e)) for e in ext]

  testingData = [cv2.imread(file) for file in files]

  count=1
  for img in testingData:
    rimg = resize(img)
    rimg.save('/content/drive/MyDrive/nature_12K/resized/test/'+name+'/'+str(count)+'.jpg')
    count+=1

  
  end_time = datetime.now()
  print('Duration: {}'.format(end_time - start_time))

trainingData = np.array(trainingData)
testingData  = np.array(testingData)

In [ ]:
# Data Loading


import cv2
import glob

trainingData = []
trainingLabels = []
validationData = []
validationLabels = []

folders = ['Animalia', 'Aves', 'Amphibia', 'Arachnida', 'Plantae', 'Fungi', 'Mollusca', 'Mammalia', 'Insecta', 'Reptilia']
classes = { folders[i] : i for i in range(0,len(folders))}

print("=========== Training Images=====================")
for name in folders:
  print(name)
  start_time = datetime.now()
  imdir = '/content/drive/MyDrive/nature_12K/resized/train/'+name+'/'
  ext = ['png', 'jpg', 'gif']    # Add image formats here

  files = []
  [files.extend(glob.glob(imdir + '*.' + e)) for e in ext]

  tempTrainingImages = [cv2.imread(file) for file in files]
  tempTrainingLabels = [classes[name] for i in range(0,len(files))]

  tempTrainingImages = np.array(tempTrainingImages,dtype=object)
  tempTrainingLabels = np.array(tempTrainingLabels)
  

  x_train, x_val, y_train, y_val = train_test_split(tempTrainingImages, tempTrainingLabels, test_size=0.1, random_state=33)

  trainingData = trainingData + list(x_train)
  trainingLabels = trainingLabels + list(y_train)

  validationData = validationData + list(x_val)
  validationLabels = validationLabels + list(y_val)
  
  end_time = datetime.now()
  print('-> Duration: {}'.format(end_time - start_time))


print("=========== Testing Images=====================")
testingData = []
testingLabels = []
for name in folders:
  print(name)
  start_time = datetime.now()
  imdir = '/content/drive/MyDrive/nature_12K/resized/test/'+name+'/'
  ext = ['png', 'jpg', 'gif']    # Add image formats here

  files = []
  [files.extend(glob.glob(imdir + '*.' + e)) for e in ext]

  testingData = testingData + [cv2.imread(file) for file in files]
  testingLabels = testingLabels + [classes[name] for i in range(0,len(files))]

  end_time = datetime.now()
  print('-> Duration: {}'.format(end_time - start_time))

trainingData = np.array(trainingData,dtype=object)
validationData = np.array(validationData,dtype=object)
testingData  = np.array(testingData,dtype=object)

In [ ]:
#Create dictionary for class indexes
train_data_path = os.path.join(actual_data_path, "train")
classes = [] #to store class values
for data_path in glob.glob(train_data_path + "/*"):
    classes.append(data_path.split('/')[-1])
idx_to_class = {i:j for i, j in enumerate(classes)} #index to class map
class_to_idx = {value:key for key,value in idx_to_class.items()} #class to index map

In [ ]:
# checking if gpu is available or not
Device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.get_device_name(0))
print(Device)

In [ ]:
# get all the paths from train_data_path and returns image paths for train and validation set
def CreateTrainDataset(actual_data_path):
    train_data_path = os.path.join(actual_data_path, "train")
    train_image_paths = [] #to store image paths in list
    classes = [] #to store class values
    for data_path in glob.glob(train_data_path + "/*"):
        train_image_paths.append(glob.glob(data_path + '/*')) #stores all the training image paths in this list
    train_image_paths = list(chain.from_iterable(train_image_paths))
    random.shuffle(train_image_paths)

    # split train valid from train paths (90,10)
    train_image_paths, valid_image_paths = train_image_paths[:int(0.9*len(train_image_paths))], train_image_paths[int(0.9*len(train_image_paths)):] 
    return train_image_paths, valid_image_paths

# create the test_image_paths
def CreateTestDataset(actual_data_path):
    test_data_path = os.path.join(actual_data_path, "val")
    test_image_paths = []
    for data_path in glob.glob(test_data_path + '/*'):
        test_image_paths.append(glob.glob(data_path + '/*')) #stores all the test images path in this list
    test_image_paths = list(chain.from_iterable(test_image_paths))
    return test_image_paths

In [ ]:
#Function returns images and corresponding lebels after performing transforms
class iNaturalist_12KDataset(Dataset):
    def __init__(self, image_paths, transform=False):
        self.image_paths = image_paths
        self.transform = transform
        
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_filepath = self.image_paths[idx]
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        label = image_filepath.split('/')[-2]
        label = class_to_idx[label]
        if self.transform is not None:
            image = self.transform(image=image)["image"]
        return image, label
    
# all input images are resized reshized_shape   
resized_shape = 256

# Data augumentation transform functions
augmented_transforms = A.Compose([A.SmallestMaxSize(max_size=350),
              A.Resize(resized_shape,resized_shape),
              A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=360, p=0.5),
              A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
              A.RandomBrightnessContrast(p=0.5),
              A.MultiplicativeNoise(multiplier=[0.5,2], per_channel=True, p=0.2),
              A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
              A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
              A.RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
              ToTensorV2()])

In [ ]:
# train_model will train the given pretrained model with TrainDataset with num_epochs and return the best model with weights 
def train_model(model, criterion, optimizer, num_epochs, batch_size, TrainDataset, ValDataset):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, int(np.ceil(len(DataLoader(TrainDataset, batch_size,shuffle=True).dataset)/batch_size)))
    
    for epoch in range(num_epochs):
        epochStartTime = time.time()
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        datasize = 0

        model.train()
        running_loss = 0.0
        running_corrects = 0

        for data in DataLoader(TrainDataset, batch_size,shuffle=True):
            inputs,labels = data
            inputs = inputs.to(Device)
            labels = labels.to(Device)
            optimizer.zero_grad()
            
            with torch.set_grad_enabled(True):
                model=model.to(Device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                loss.backward()
                optimizer.step()
                scheduler.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                datasize += inputs.shape[0]

        training_epoch_loss = running_loss / datasize
        training_epoch_acc = running_corrects.double() / datasize

        print('{} Loss: {:.4f} Acc: {:.4f}'.format('Train', training_epoch_loss, training_epoch_acc))

        datasize = 0
        model.eval()
        running_loss = 0.0
        running_corrects = 0

        for data in DataLoader(ValDataset,batch_size):
            inputs,labels = data
            inputs = inputs.to(Device)
            labels = labels.to(Device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(False):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            datasize += inputs.shape[0]

        validation_epoch_loss = running_loss / datasize
        validation_epoch_acc = running_corrects.double() / datasize

        print('{} Loss: {:.4f} Acc: {:.4f}'.format('Val', validation_epoch_loss, validation_epoch_acc))

        if validation_epoch_acc > best_acc:
            best_acc = validation_epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
        print()

        wandb.log({
        "epoch": num_epochs,
        "loss": training_epoch_loss,
        "accuracy": training_epoch_acc,
        "val_loss": validation_epoch_loss,
        "val_accuracy": validation_epoch_acc})
        
        epochEndTime = time.time() - epochStartTime
        print('Epoch complete in {:.0f}m {:.0f}s'.format(
                epochEndTime // 60, epochEndTime % 60))
            

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
                time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

        # load best model weights
    model.load_state_dict(best_model_wts)
    return model

def test_model(model, TestDataset,batch_size):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in DataLoader(TestDataset,batch_size):
            images, labels = data
            images = images.to(Device)
            labels = labels.to(Device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the test images: {100 * correct // total} %')


In [ ]:
def wandDBTraining(config=None):
    resized_shape = 256
    dataset_augmentation = True
    
    train_transforms = A.Compose([A.Resize(resized_shape,resized_shape),A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),ToTensorV2()])
    train_image_paths, valid_image_paths=CreateTrainDataset(actual_data_path)
    TrainDataset = iNaturalist_12KDataset(train_image_paths,train_transforms)
    
    if dataset_augmentation:
        transformed_dataset = iNaturalist_12KDataset(train_image_paths,augmented_transforms)   #Transformed Dataset created with augmented_transforms
        TrainDataset = torch.utils.data.ConcatDataset([transformed_dataset,TrainDataset])
    
    ValDataset = iNaturalist_12KDataset(valid_image_paths,train_transforms)
    test_transforms = A.Compose([A.Resize(resized_shape,resized_shape),A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),ToTensorV2()])
    test_image_paths=CreateTestDataset(actual_data_path)
    TestDataset = iNaturalist_12KDataset(test_image_paths,test_transforms)
    
    with wandb.init(config=config, ):
        config=wandb.config
        wandb.run.name = 'm'+str(config.model)+'-fre-'+ str(config.unfreezed_from_last)+'-bs-'+str(config.batch_size)+ '-lr-'+str(config.learning_rate)+ '-ep-'+str(config.epochs) +'-wd-'+str(config.weight_decay)

        if config.model == 'InceptionV3':
            model = models.inception_v3(pretrained=True,aux_logits=False)
        elif config.model == 'resnet50':
            model = models.resnet50(pretrained=True)
        elif config.model == 'resnet18':
            model = models.resnet18(pretrained=True)
        elif config.model == 'googlenet':
            model = models.googlenet(pretrained=True,aux_logits=False)
        elif config.model == 'InceptionResNetV2':
            model = timm.create_model('inception_resnet_v2', pretrained=True)

        for param in list(model.parameters())[: -(config.unfreezed_from_last+2)]:
            param.requires_grad = False

        if config.model == 'InceptionResNetV2':
            num_ftrs = model.classif.in_features
            model.classif = nn.Linear(num_ftrs, 10)
        else:
            num_ftrs = model.fc.in_features
            model.fc = nn.Linear(num_ftrs, 10)
        
        criterion = nn.CrossEntropyLoss()
        optimizer_ft = optim.Adam(model.parameters(), lr=config.learning_rate,weight_decay = config.weight_decay)

        model = train_model(model, criterion, optimizer_ft, config.epochs, config.batch_size, TrainDataset, ValDataset)
        test_model(model, TestDataset, config.batch_size)

In [ ]:
sweep_config = {
    "method":"bayes",
}
metric = {
    "name" : "val_accuracy",
    "goal" : "maximize"
}
sweep_config['metric']=metric
parameter_dict = {
        'model': {
            'values': ["InceptionResNetV2"]
        },
        'unfreezed_from_last': {
            'values': [2]
        },
        'batch_size': {
            'values': [8]
        },
        'learning_rate': {
            'values': [0.002]
        },
        'epochs': {
            'values': [10]
        },
         'weight_decay': {
            'values': [0.0002]
        }
}

sweep_config['parameters']=parameter_dict
import pprint

pprint.pprint(sweep_config)
sweep_id = wandb.sweep(sweep_config,project="Assignment2_Testing")

In [ ]:
wandb.agent(sweep_id, wandDBTraining, count=1)